In [7]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.linalg as spla

def dg(A):
    return A.conj().T

In [5]:
# Model parameters.
γ = 1
δ = 0
Ω = 0.5
dΩ = 0.001

# Initial state.
ψ0 = np.array([1, 0], dtype="complex128")

# Simulation parameters.
number_trajectories = 10
t_final = 100
dt = 0.01

In [6]:
# Operators.
σ = np.array([[0,1],[0,0]], dtype="complex128")
c = np.sqrt(γ) * σ
σz = np.array([[1,0], [0,-1]], dtype="complex128")

# Hamiltonian and jump operator.
H = - δ * dg(σ) @ σ + Ω/2. * (dg(σ) + σ)
M1 = np.sqrt(dt) * c

# Displaced operators to compute the derivatives.
Hp = - δ * dg(σ) @ σ + (Ω + dΩ)/2. * (dg(σ) + σ)
Hm = - δ * dg(σ) @ σ + (Ω - dΩ)/2. * (dg(σ) + σ)
M1p = M1
M1m = M1

# The jump operators have to be passed as a list.
M1_l = [M1]
M1p_l = [M1p]
M1m_l = [M1m]

In [8]:
def gillespie_fisher(H, Hp, Hm, M_l, Mp_l, Mm_l, dθ, ψ0, t_final, dt, number_trajectories):
    print("Gillespie-Fisher method - Welcome")
    print("=> Starting the pre-computation stage")
    t_range = np.arange(0, t_final, dt)
    
    # Constructs the overall jump operator.
    J = np.zeros_like(M_l[0])
    for M in M_l:
        J += dg(M) @ M
    # Effective (non-Hermitian) Hamiltonian.
    He = H - 1j/2. * J
    
    # Constructs the no-jump evolution operators for all the relevant times.
    V = [] # List of the no-jump evolution operators.
    Qs = [] # List of the non-state-dependent part of the waiting time distribution.
    for t in t_range:
        ev_op = spla.expm(-1j * He * t)
        V.append(ev_op)
        nsd_wtd = dg(ev_op) @ J @ ev_op
        Qs.append(nsd_wtd)
    # Prints the matrix norm of the latest Qs.
    error = spla.norm(Qs[-1])
    print(f"-> Truncation error given by norm of latest Qs matrix -> {error}")
    
    # Displaced versions of the effective Hamiltonian.
    Jp = np.zeros_like(Mp_l[0])
    for Mp in Mp_l:
        Jp += dg(Mp) @ Mp
    Hep = Hp - 1j/2. * Jp
    Jm = np.zeros_like(Mm_l[0])
    for Mm in Mm_l:
        Jm += dg(Mm) @ Mm
    Hem = Hm - 1j/2. * Jm
    
    # Vector of the derivatives of the evolution operator wrt the parameter.
    Vdot = []
    for t in t_range:
        vd = (spla.expm(-1j * Hep * t) - spla.expm(-1j * Hem * t)) / (2. * dθ)
        Vdot.append(vd)
    # Derivatives of all the jump operators.
    Mdot = []
    for n_M in range(len(M_l)):
        Mdot.append((Mp_l[n_M] - Mm_l[n_M]) / (2 * dθ))
    # Precomputation of all the derivatives of MkV(t).
    Δ = []
    for n_M in range(len(M_l)):
        fixed_M = []
        for n_t, t in enumerate(t_range):
            obj = Mdot[n_M] @ V[n_t] + M[n_M] @ Vdot[n_t]
            fixed_M.append(obj)
        Δ.append(fixed_M)
    
    # Evolution cycle.
    print("=> Starting the evolution cycle.")
    # Cycle over the trajectories.
    for trajectory in range(number_trajectories):
        print(f"-> Starting trajectory {trajectory}")
        